In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
### Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from  sklearn.model_selection import train_test_split , StratifiedKFold

import warnings 
warnings.simplefilter('ignore')

#### Read Data Files 


In [ ]:
marketing_campaign = pd.read_csv('/kaggle/input/arketing-campaign/marketing_campaign.csv' , header=0,sep=";")
marketing_campaign.shape

In [ ]:
marketing_campaign.head()

### Lets find the basic details of the data with respect to data quality issues and descriptive statistics

In [ ]:
print(marketing_campaign.describe())
print('-----------------------------------------------')
print(marketing_campaign.info())

#### Find Categorical and Numerical variables from the dataset

In [ ]:
data = marketing_campaign.copy()

Dt_Customer is object type but if we see the data it is actually date field. Lets convert it to date datatype

In [ ]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])

In [ ]:
### Find Numerical and Categorical features
numerical = [features for features in data.columns if data[features].dtypes != 'O'] 

categorical = [features for features in data.columns if data[features].dtypes == 'O']

print('Numerical Features are : ')
print(numerical)
print('Categorical Features are : ')
print(categorical)

Purpose is to find the features which are impacting the acceptance of campaign by customer.
There are customer details which may affect the responce. Lets check the relation between the features.

### Data Analysis

#### Categorical features : Univariate analysis

In [ ]:
def CatBar(data,feature):
    fig = data[feature].value_counts().plot.bar()
    fig.set_title(feature)
    fig.set_ylabel('Number')
    plt.show()

In [ ]:
for f in categorical:
    CatBar(data,f)

#### Categorical features : Bivariate analysis

In [ ]:
pd.crosstab(index= data['Education'] , columns=data['Marital_Status']).plot(kind="bar", 
                 figsize=(8,8),
                 stacked=True)

Lets check categorical features relation with Response numerical feature.
Also lets see who has responded in which attempt

In [ ]:
pd.crosstab(index= data['Education'] , columns=data['Response']).plot(kind="bar", 
                 figsize=(8,8),
                 stacked=True)

In [ ]:
pd.crosstab(index= data['Marital_Status'] , columns=data['Response']).plot(kind="bar", 
                 figsize=(8,8),
                 stacked=True)

### Observations -->
1. Education as Basic or 2nd Cycle have not responded to any camaign
2. Proportion of response of Phd is higher than Bachalor or Masters educated 
3. proporation of response is better if we consider the data as is than married. 
However , if we consider Widow , Together , Divorced as Married (As couple is staying together or was together) then this percentage is higher for married

#### Lets further drill down to check , from the responses , how many are responded in first , second or last attemp etc. To do that lets create data subset with positive response

In [ ]:
data_y = data[data.Response == 1]
data_y.shape

### Lets check for first attempt

In [ ]:
pd.crosstab(index= data_y['AcceptedCmp1'] , columns=data_y['Marital_Status'])

### Observations 
1. Divorced (14%), Together (16%)   -- >  less than or around 20% responded in 1st attempt
2. Single -- > 23% of Single has responded in first attempt
3. Married -- > 32% of Married have responded in first attempt
4. Widow -- > 21% responded in first attempt

In [ ]:
pd.crosstab(index= data_y['AcceptedCmp1'] , columns=data_y['Education'])

### Observation -->
1. 2nd cycle --> 31% responded in 1st attempt
2. Graduation -- > 27% responded in 1st attempt
3. Master -- > 17% responded in 1st attemp1 
4. Phd -- > 20% responded in 1st attempt 

### Lets check for Second attempt

In [ ]:
def create_cross(data,feature,target):
    x = pd.crosstab(data[feature] , data[target])
    print(x)
                    

In [ ]:
features = ['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5']
for f in features:
    print(f)
    print('-------------------------------------------------------------')
    create_cross(data,f , 'Education')
    print('-------------------------------------------------------------')

### Observation -->
1. 2nd cycle --> 31% responded in 1st attempt , almost 0% responded in 2nd , 7% in 3rd, 4% in 4th , 4% in 5th
2. Basic -- > 0% in 1st , 0% in second , 11% in 3rd , 0% in 4th , 0% in 5th 
3. Graduation -- > 7% responded in 1st attempt, 1.4% in 2nd , 7% in 3rd , 7% in 4tbh, 7% in 5th
**( Graduates has equal chance of responding so multiple attempts are useful)**
4. Master -- >  4% in 1st , 0% in 2nd , 10% in 3rd , 12% in 4th , 10% in 5th 
**(Masters will respond better if we reach out to them multiple times)**
5. Phd -- > 6% responded in 1st attemp1, 2% in 2nd , 8% in 3rd , 9% in 4th , 8% in 5th
**( Phd have equal chance of responding so multiple attempts are useful)**

In [ ]:
for f in features:
    print(f)
    print('-------------------------------------------------------------')
    create_cross(data,f , 'Marital_Status')
    print('-------------------------------------------------------------')

### Observation -->
1. Absurd -- > This seems to be DQ issue
2. Alone -- > 50% change in 3rd attempt
3. Divorced -- > 
4. Married 
5. Single 
6. Together
7. Widow 
8. YOLO

##### Numerical Features : Univariate Analysis

In [ ]:
data_y.head()

In [ ]:
for f in numerical:
    sns.distplot(data_y[f])
    plt.show()

Instead of Year_Birth , we can convert it to Age of customer and then check the relationship with Response

In [ ]:
data['Age'] = 2019 - data['Year_Birth']
data['Age'].head()
#data['Year_Birth'].min()

In [ ]:
### Lets check relation of customer Age with Response
data_y['Age'] = 2019 - data_y['Year_Birth']

In [ ]:
x = data[data['Response'] == 1]

x_30 = x[x['Age'] <=30]
x_60 = x[(x['Age'] >30) & (x['Age'] <=60) ]
X_100 = x[x['Age'] > 60]

In [ ]:
x_30.groupby('Age')['ID'].count().plot(kind='bar')

In [ ]:
x_60.groupby('Age')['ID'].count().plot(kind='bar')

In [ ]:
X_100.groupby('Age')['ID'].count().plot(kind='bar')

In [ ]:
create_cross(data,'Complain' , 'Response')

Percentage wise whether customer has complained or not , it looks like the campaign success percentage is same

In [ ]:
print(data['Dt_Customer'].max())
print(data['Dt_Customer'].min())

In [ ]:
import datetime
x = pd.to_datetime(data['Dt_Customer'])
(pd.to_datetime(datetime.datetime.now().date()) - x )

In [ ]:
pd.to_datetime(datetime.datetime.now())

In [ ]:
data.columns

In [ ]:
data['NumDealsPurchases'].head()

In [ ]:
data_y.groupby('NumDealsPurchases')['ID'].count().plot(kind='bar')

In [ ]:
data_y.groupby('NumWebPurchases')['ID'].count().plot(kind='bar')

In [ ]:
data_y.groupby('NumCatalogPurchases')['ID'].count().plot(kind='bar')

In [ ]:
data_y.groupby('NumStorePurchases')['ID'].count().plot(kind='bar')